In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)


In [9]:
module = model.conv1
print(list(module.named_parameters()))


[('weight', Parameter containing:
tensor([[[[ 0.0319, -0.0283, -0.0410],
          [-0.1058,  0.0628,  0.1391],
          [ 0.2139,  0.1283, -0.0265]]],


        [[[ 0.3245, -0.2847,  0.1321],
          [ 0.0051, -0.0475, -0.0135],
          [-0.3256, -0.2450,  0.1750]]],


        [[[ 0.2834, -0.0207,  0.1597],
          [ 0.1738,  0.0266, -0.3305],
          [ 0.0215,  0.0884, -0.0436]]],


        [[[ 0.2800,  0.2257, -0.0656],
          [ 0.0157, -0.2003, -0.0971],
          [ 0.0795, -0.1221,  0.1232]]],


        [[[-0.0522, -0.1284, -0.1628],
          [ 0.0359,  0.1182,  0.1768],
          [ 0.0098,  0.3059, -0.0190]]],


        [[[-0.1305,  0.2425, -0.2519],
          [ 0.3133,  0.1959,  0.2756],
          [-0.2196,  0.0205,  0.0363]]]], device='cuda:0', requires_grad=True)), ('bias', Parameter containing:
tensor([ 0.3270,  0.0763,  0.1773, -0.0414, -0.2483, -0.1971], device='cuda:0',
       requires_grad=True))]


In [10]:
print(list(module.named_buffers()))

[]


In [11]:
prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

In [12]:
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([ 0.3270,  0.0763,  0.1773, -0.0414, -0.2483, -0.1971], device='cuda:0',
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0319, -0.0283, -0.0410],
          [-0.1058,  0.0628,  0.1391],
          [ 0.2139,  0.1283, -0.0265]]],


        [[[ 0.3245, -0.2847,  0.1321],
          [ 0.0051, -0.0475, -0.0135],
          [-0.3256, -0.2450,  0.1750]]],


        [[[ 0.2834, -0.0207,  0.1597],
          [ 0.1738,  0.0266, -0.3305],
          [ 0.0215,  0.0884, -0.0436]]],


        [[[ 0.2800,  0.2257, -0.0656],
          [ 0.0157, -0.2003, -0.0971],
          [ 0.0795, -0.1221,  0.1232]]],


        [[[-0.0522, -0.1284, -0.1628],
          [ 0.0359,  0.1182,  0.1768],
          [ 0.0098,  0.3059, -0.0190]]],


        [[[-0.1305,  0.2425, -0.2519],
          [ 0.3133,  0.1959,  0.2756],
          [-0.2196,  0.0205,  0.0363]]]], device='cuda:0', requires_grad=True))]


In [13]:
print(module.weight)

tensor([[[[ 0.0000, -0.0000, -0.0000],
          [-0.1058,  0.0628,  0.1391],
          [ 0.2139,  0.1283, -0.0000]]],


        [[[ 0.3245, -0.2847,  0.1321],
          [ 0.0000, -0.0475, -0.0000],
          [-0.3256, -0.0000,  0.0000]]],


        [[[ 0.2834, -0.0207,  0.1597],
          [ 0.1738,  0.0266, -0.3305],
          [ 0.0215,  0.0884, -0.0000]]],


        [[[ 0.2800,  0.2257, -0.0656],
          [ 0.0000, -0.2003, -0.0000],
          [ 0.0000, -0.1221,  0.1232]]],


        [[[-0.0522, -0.1284, -0.1628],
          [ 0.0359,  0.1182,  0.1768],
          [ 0.0098,  0.0000, -0.0190]]],


        [[[-0.1305,  0.0000, -0.2519],
          [ 0.3133,  0.0000,  0.2756],
          [-0.2196,  0.0000,  0.0363]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


In [14]:
print(module._forward_pre_hooks)

OrderedDict([(2, <torch.nn.utils.prune.RandomUnstructured object at 0x0000015834064040>)])


In [15]:
prune.l1_unstructured(module, name="bias", amount=3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

In [16]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 0.0319, -0.0283, -0.0410],
          [-0.1058,  0.0628,  0.1391],
          [ 0.2139,  0.1283, -0.0265]]],


        [[[ 0.3245, -0.2847,  0.1321],
          [ 0.0051, -0.0475, -0.0135],
          [-0.3256, -0.2450,  0.1750]]],


        [[[ 0.2834, -0.0207,  0.1597],
          [ 0.1738,  0.0266, -0.3305],
          [ 0.0215,  0.0884, -0.0436]]],


        [[[ 0.2800,  0.2257, -0.0656],
          [ 0.0157, -0.2003, -0.0971],
          [ 0.0795, -0.1221,  0.1232]]],


        [[[-0.0522, -0.1284, -0.1628],
          [ 0.0359,  0.1182,  0.1768],
          [ 0.0098,  0.3059, -0.0190]]],


        [[[-0.1305,  0.2425, -0.2519],
          [ 0.3133,  0.1959,  0.2756],
          [-0.2196,  0.0205,  0.0363]]]], device='cuda:0', requires_grad=True)), ('bias_orig', Parameter containing:
tensor([ 0.3270,  0.0763,  0.1773, -0.0414, -0.2483, -0.1971], device='cuda:0',
       requires_grad=True))]


In [17]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 0., 0.],
          [1., 1., 1.],
          [1., 1., 0.]]],


        [[[1., 1., 1.],
          [0., 1., 0.],
          [1., 0., 0.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 0.]]],


        [[[1., 1., 1.],
          [0., 1., 0.],
          [0., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 0., 1.],
          [1., 0., 1.],
          [1., 0., 1.]]]], device='cuda:0')), ('bias_mask', tensor([1., 0., 0., 0., 1., 1.], device='cuda:0'))]


In [18]:
print(module.bias)

tensor([ 0.3270,  0.0000,  0.0000, -0.0000, -0.2483, -0.1971], device='cuda:0',
       grad_fn=<MulBackward0>)


In [19]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# As we can verify, this will zero out all the connections corresponding to
# 50% (3 out of 6) of the channels, while preserving the action of the
# previous mask.
print(module.weight)

tensor([[[[ 0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000]]],


        [[[ 0.3245, -0.2847,  0.1321],
          [ 0.0000, -0.0475, -0.0000],
          [-0.3256, -0.0000,  0.0000]]],


        [[[ 0.2834, -0.0207,  0.1597],
          [ 0.1738,  0.0266, -0.3305],
          [ 0.0215,  0.0884, -0.0000]]],


        [[[ 0.0000,  0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000]]],


        [[[-0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000]]],


        [[[-0.1305,  0.0000, -0.2519],
          [ 0.3133,  0.0000,  0.2756],
          [-0.2196,  0.0000,  0.0363]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


In [20]:
new_model = LeNet()
for name, module in new_model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

print(dict(new_model.named_buffers()).keys())  # to verify that all masks exist


dict_keys(['conv1.weight_mask', 'conv2.weight_mask', 'fc1.weight_mask', 'fc2.weight_mask', 'fc3.weight_mask'])


In [21]:
model = LeNet()

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)


Sparsity in conv1.weight: 1.85%
Sparsity in conv2.weight: 6.83%
Sparsity in fc1.weight: 22.11%
Sparsity in fc2.weight: 12.08%
Sparsity in fc3.weight: 9.17%
Global sparsity: 20.00%


In [ ]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0
        return mask
def foobar_unstructured(module, name):
    """Prunes tensor corresponding to parameter called `name` in `module`
    by removing every other entry in the tensors.
    Modifies module in place (and also return the modified module)
    by:
    1) adding a named buffer called `name+'_mask'` corresponding to the
    binary mask applied to the parameter `name` by the pruning method.
    The parameter `name` is replaced by its pruned version, while the
    original (unpruned) parameter is stored in a new parameter named
    `name+'_orig'`.

    Args:
        module (nn.Module): module containing the tensor to prune
        name (string): parameter name within `module` on which pruning
                will act.

    Returns:
        module (nn.Module): modified (i.e. pruned) version of the input
            module

    Examples:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)
    return module
model = LeNet()
foobar_unstructured(model.fc3, name='bias')

print(model.fc3.bias_mask)
